In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import pandas as pd
import seaborn as sns
from scipy import stats

In [2]:
preproceeed_data_path = "../dataset/Nakano_etal_2010/preprocessed_data"
df = pd.read_csv(f"{preproceeed_data_path}/eyeclip_annotation.csv")
obj_pos_upsample = np.load(f"{preproceeed_data_path}/obj_pos_upsample.npz")["data"]
num_clip = df["clip_index"].max()
object_labels = ["obj", "eye", "mouth", "nose", "ear", "face", "hand"]
num_obj_type = len(object_labels)

### Subj

In [3]:
gaze_weight_subj = np.load(f"{preproceeed_data_path}/gaze_weight_goodsubj.npz", allow_pickle=True)
gaze_weight_goodsubj = gaze_weight_subj['gaze_weight_goodsubj']
gr_goodsubj = gaze_weight_subj['gr_goodsubj']
group_name = gaze_weight_subj['group_name']
num_goodsubj = len(gr_goodsubj)
gr_indices = [0, 1, 4, 3]

In [4]:
gazew_parts_goodsubj = np.zeros((num_goodsubj, num_clip, num_obj_type))
for clip_idx in tqdm(range(num_clip)):
    df_clip = df[df["clip_index"] == (clip_idx+1)].reset_index()
    sidx = df_clip["start_resampled_index"].unique()[0]
    eidx = df_clip["end_resampled_index"].unique()[0]

    for i, obj_l in enumerate(object_labels):
        obj_indices = df_clip[df_clip[obj_l] > 0].index
        if len(obj_indices) > 0:
            for obj_idx in obj_indices:
                obj_exist = np.sum(~np.isnan(obj_pos_upsample[:, obj_idx, sidx:eidx]), axis=0) > 1
                obj_exist_sum = np.sum(obj_exist)
                if obj_exist_sum > 0:
                    for subj_idx in range(num_goodsubj):
                        gazew_clip_obj_subj = gaze_weight_goodsubj[subj_idx, sidx:eidx, obj_idx]
                        gazew_parts_goodsubj[subj_idx, clip_idx, i] += np.sum(gazew_clip_obj_subj[obj_exist]) / obj_exist_sum * 100
        else:
            gazew_parts_goodsubj[:, clip_idx, i] = np.nan

  0%|          | 0/12 [00:00<?, ?it/s]

In [5]:
gazew_parts_goodsubj_dict = {}
gazew_parts_goodsubj_dict["gazew_parts"] = {}
for gr_idx in gr_indices:
    gazew_parts_goodsubj_dict["gazew_parts"][group_name[gr_idx]] = gazew_parts_goodsubj[gr_goodsubj == gr_idx]
gazew_parts_goodsubj_dict["gr_keys"] = [group_name[gr_idx] for gr_idx in gr_indices]
gazew_parts_goodsubj_dict["object_labels"] = object_labels

In [6]:
np.savez_compressed("../results/gazew_parts_goodsubj_N2010.npz", **gazew_parts_goodsubj_dict)

### ViT

In [7]:
gaze_weight_vit = np.load(f"{preproceeed_data_path}/gaze_weight_vit.npz", allow_pickle=True)

In [8]:
gaze_weight_vit.files

['dino', 'supervised', 'dino_deit_small16', 'supervised_deit_small16']

In [9]:
training_methods = ["dino", "supervised"] 
depth_list = [4, 8, 12]
num_models, _, _, num_sampling, _ = gaze_weight_vit["dino"].item()["4"].shape
num_heads = 6

In [10]:
gazew_parts_vit_dict = {}
gazew_parts_vit_dict["gazew_parts"] = {}
for tm in training_methods:
    gazew_parts_vit_dict["gazew_parts"][tm] = {}
    for depth in depth_list:
        gazew_parts_vit_depth = np.zeros((num_models, depth, num_heads, num_clip, num_obj_type))
        gaze_weight_vit_depth = gaze_weight_vit[tm].item()[str(depth)]
        for clip_idx in tqdm(range(num_clip)):
            df_clip = df[df["clip_index"] == (clip_idx+1)].reset_index()
            sidx = df_clip["start_resampled_index"].unique()[0]
            eidx = df_clip["end_resampled_index"].unique()[0]
        
            for i, obj_l in enumerate(object_labels):
                obj_indices = df_clip[df_clip[obj_l] > 0].index
                if len(obj_indices) > 0:
                    for obj_idx in obj_indices:
                        obj_exist = np.sum(~np.isnan(obj_pos_upsample[:, obj_idx, sidx:eidx]), axis=0) > 1
                        obj_exist_sum = np.sum(obj_exist)
                        if obj_exist_sum > 0:
                            for m_idx in range(num_models):
                                for d_idx in range(depth):
                                    for h_idx in range(num_heads):
                                        gazew_obj_head = gaze_weight_vit_depth[m_idx, d_idx, h_idx, sidx:eidx, obj_idx]
                                        gazew_parts_vit_depth[m_idx, d_idx, h_idx, clip_idx, i] += np.sum(gazew_obj_head[obj_exist]) / obj_exist_sum * 100
                else:
                    gazew_parts_vit_depth[:, :, :, clip_idx, i] = np.nan
        gazew_parts_vit_dict["gazew_parts"][tm][str(depth)] = gazew_parts_vit_depth

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

In [11]:
for tm in training_methods:
    gazew_parts_vit_key = np.zeros((depth, num_heads, num_clip, num_obj_type))
    gaze_weight_vit_key = gaze_weight_vit[f'{tm}_deit_small16']
    depth, _, _, _ = gaze_weight_vit_key.shape
    for clip_idx in tqdm(range(num_clip)):
        df_clip = df[df["clip_index"] == (clip_idx+1)].reset_index()
        sidx = df_clip["start_resampled_index"].unique()[0]
        eidx = df_clip["end_resampled_index"].unique()[0]
    
        for i, obj_l in enumerate(object_labels):
            obj_indices = df_clip[df_clip[obj_l] > 0].index
            if len(obj_indices) > 0:
                for obj_idx in obj_indices:
                    obj_exist = np.sum(~np.isnan(obj_pos_upsample[:, obj_idx, sidx:eidx]), axis=0) > 1
                    obj_exist_sum = np.sum(obj_exist)
                    if obj_exist_sum > 0:
                        for d_idx in range(depth):
                            for h_idx in range(num_heads):
                                gazew_obj_head = gaze_weight_vit_key[d_idx, h_idx, sidx:eidx, obj_idx]
                                gazew_parts_vit_key[d_idx, h_idx, clip_idx, i] += np.sum(gazew_obj_head[obj_exist]) / obj_exist_sum * 100
            else:
                gazew_parts_vit_key[:, :, clip_idx, i] = np.nan
    gazew_parts_vit_dict["gazew_parts"][f'{tm}_deit_small16'] = gazew_parts_vit_key

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

In [12]:
gazew_obj_head.shape

(241,)

In [13]:
gazew_parts_vit_dict["object_labels"] = object_labels

In [14]:
np.savez_compressed("../results/gazew_parts_vit_N2010.npz", **gazew_parts_vit_dict)

### GBVS

In [15]:
gaze_weight_gbvs_data = np.load(f"{preproceeed_data_path}/gaze_weight_gbvs.npz", allow_pickle=True)
gaze_weight_gbvs = gaze_weight_gbvs_data['all_data']
gbvs_labels = gaze_weight_gbvs_data["labels"]
num_gbvs = len(gbvs_labels)

In [16]:
gazew_parts_gbvs_dict = {}
gazew_parts_gbvs = np.zeros((num_gbvs, num_clip, num_obj_type))
for clip_idx in tqdm(range(num_clip)):
    df_clip = df[df["clip_index"] == (clip_idx+1)].reset_index()
    sidx = df_clip["start_resampled_index"].unique()[0]
    eidx = df_clip["end_resampled_index"].unique()[0]

    for i, obj_l in enumerate(object_labels):
        obj_indices = df_clip[df_clip[obj_l] > 0].index
        if len(obj_indices) > 0:
            for obj_idx in obj_indices:
                obj_exist = np.sum(~np.isnan(obj_pos_upsample[:, obj_idx, sidx:eidx]), axis=0) > 1
                obj_exist_sum = np.sum(obj_exist)
                if obj_exist_sum > 0:
                    for key_idx in range(num_gbvs):
                        gazew_obj_head = gaze_weight_gbvs[key_idx, sidx:eidx, obj_idx]
                        gazew_parts_gbvs[key_idx, clip_idx, i] += np.sum(gazew_obj_head[obj_exist]) / obj_exist_sum * 100
        else:
            gazew_parts_gbvs[:, clip_idx, i] = np.nan
gazew_parts_gbvs_dict["gazew_parts"] = gazew_parts_gbvs
gazew_parts_gbvs_dict["gbvs_labels"] = gbvs_labels
gazew_parts_gbvs_dict["object_labels"] = object_labels

  0%|          | 0/12 [00:00<?, ?it/s]

In [17]:
np.savez_compressed("../results/gazew_parts_gbvs_N2010.npz", **gazew_parts_gbvs_dict)